<a href="https://colab.research.google.com/github/AbdulWahabRaza123/NLP/blob/main/FastTextNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is also one of the most popular and fast technique to convert text to numbers. Here we have 2 type**

1.   CBow (Continues beg of words) (Here we have context and we try to find the target word e.g we have ordered of is context and target word may be a king)
2.   skipGram (this is inverse of cbow here we have target word and we will try to find the context)


**Note:** In word2vec we will train model on words but in fast text we will use the charecters here we have n which is the number of charecter which we will use for training 

==>**Word2Vec can cause out of vocabulary problem but fast text can tackle with this problem**

==>**Here we have custom word embeddings of any domain**

==>**Fast Text is technique as well as library**

In [2]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4402191 sha256=7dd4759a770b59a78e476266510f84373c061450f25360ede415a5e16e07fe13
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [3]:
import fasttext

In [4]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/codebasics/nlp-tutorials/main/18_fasttext_classification/ecommerce_dataset.csv",names=["category","description"],header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [6]:
df.category.value_counts()

Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: category, dtype: int64

In [9]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [11]:
df.category.replace("Clothing & Accessories","Clothing_Accessories",inplace=True)
df.category.value_counts()

Household               19313
Books                   11820
Electronics             10621
Clothing_Accessories     8670
Name: category, dtype: int64

**Note:** There is a rule for supplying label to fast text which is we have to put `__label__` for every category and if there is seperation then seperate the text with _ case

In [12]:
df["category"]="__label__"+df["category"].astype(str)
df.head()

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


**Note:** There is also a rule that to train fast text model we have give the file to model in specific formate where full data should be in a single line

In [23]:
df['category_description']=df.category+" "+df.description
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


**Note:** Now removing punctuations etc. by doing preprocessing

In [24]:
import re
def preprocess(text):
  re.sub(r'[^\w\s\']',' ',text)
  text=re.sub(r' +',' ',text)
  #to remove starting spaces and convert to lower case
  return text.strip().lower()

In [25]:
preprocess("  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi")

"viki's | bookcase/bookshelf (3-shelf/shelve, white) | ? . hi"

In [26]:
df['category_description']=df['category_description'].map(preprocess)

In [27]:
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [29]:
#now to start trainning use train_test_split
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.2)

In [30]:
train.shape,test.shape

((40339, 3), (10085, 3))

In [31]:
train.head()

,category,description,category_description
48583,__label__Electronics,Philips AJ3400/37 Clock Radio (Black) It has a...,__label__electronics philips aj3400/37 clock r...
16851,__label__Household,Dolphy Multifold Mini Hand Towel Paper Dispens...,__label__household dolphy multifold mini hand ...
42733,__label__Electronics,Seagate 2TB Expansion USB 3.0 Portable 2.5 Inc...,__label__electronics seagate 2tb expansion usb...
33842,__label__Clothing_Accessories,Elk Kids Baby Girls Printed Panties Brief Inne...,__label__clothing_accessories elk kids baby gi...
28177,__label__Books,The Government Class Book: A Youth's Manual of...,__label__books the government class book: a yo...


In [32]:
test.head()

,category,description,category_description
14173,__label__Household,YFXOHAR Travel Garment Handheld Portable Plast...,__label__household yfxohar travel garment hand...
35904,__label__Clothing_Accessories,Arrow Men's Printed Regular Fit Formal Shirt M...,__label__clothing_accessories arrow men's prin...
15711,__label__Household,Citra LED Strip Light Waterproof Roll 14 Meter...,__label__household citra led strip light water...
48120,__label__Electronics,Woodman BT10 10-inch Bass Tube with in-Built A...,__label__electronics woodman bt10 10-inch bass...
42793,__label__Electronics,Logitech MK200 Media Wired Keyboard and Mouse ...,__label__electronics logitech mk200 media wire...


In [36]:
#now storing file for fasttext trainning
train.to_csv("ecommerce.train",columns=["category_description"],index=False,header=False)

In [37]:
test.to_csv("ecommerce.test",columns=["category_description"],index=False,header=False)

In [38]:
import fasttext

In [39]:
#to do the classification using fast text
model=fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(2302, 0.9639443961772372, 0.9639443961772372)

In [40]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.97724229]))

In [41]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([0.99999833]))

In [42]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([0.99998224]))

In [43]:
model.get_nearest_neighbors("painting")

[(0.7277058959007263, 'beauty'),
 (0.7192052006721497, 'si'),
 (0.719180166721344, 'study'),
 (0.7162912487983704, '(translation)'),
 (0.7160165309906006, '2200'),
 (0.7139139175415039, '-7'),
 (0.7075243592262268, 'living'),
 (0.7073867321014404, 'solutions'),
 (0.705409049987793, 'grade'),
 (0.6985952258110046, 'strauss')]

In [44]:
model.get_nearest_neighbors("sony")

[(0.9994878768920898, '15-e'),
 (0.9994878768920898, '15-s'),
 (0.9994878768920898, '15-n'),
 (0.9994878768920898, '15-g'),
 (0.9994878768920898, '\xa0hp'),
 (0.9994878768920898, '15-r'),
 (0.9994878768920898, 'series(numeric)'),
 (0.9994768500328064, 'vostro'),
 (0.9993700385093689, 'l210/'),
 (0.9993700385093689, 'l220/')]

In [45]:
model.get_nearest_neighbors("islamabad")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'aditii'),
 (0.0, 'declaration.'),
 (0.0, 'truth."')]